In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install tsplib95
import numpy as np
import pandas as pd
import networkx as nx
import tsplib95
import matplotlib.pyplot as plt
import math
import random
from random import sample
import pickle
from scipy.spatial import distance
import time

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input/tsp-problems/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

np.random.seed(22)
random.seed(22)

In [2]:
def generate_random_inividual(G, dist_type="EUC_2D"):
    path = generate_random_path(G)
    return individual(G, get_edge_list(path),dist_type)

def generate_random_population(G,pop_size=10, dist_type="EUC_2D"):
    return population(G,[generate_random_inividual(G,dist_type) for i in range(pop_size)])

def mutate(path, mutation_rate=None):
    if mutation_rate==None: mutation_rate=1/len(path)

    for i in range(len(path)):
        if np.random.random()>mutation_rate:
            next
        else:
            path = swap_positions_path(path,i)
    return path

def crossover(parent1, parent2, mutation_rate=0.1):
    child1 = []
    child2 = []
    cuts = [np.random.randint(len(parent1)), np.random.randint(len(parent1))]
    
    for i in range(min(cuts), max(cuts)):
        child1.append(parent1[i])

    child1 = child1 + [i for i in parent2 if i not in child1]
    child1=mutate(child1,mutation_rate)

    for i in range(min(cuts), max(cuts)):
        child2.append(parent1[i])

    child2 = child2 + [i for i in parent2 if i not in child2]
    child2=mutate(child2,mutation_rate)

    return np.array(child1), np.array(child2)

def split_path(G,original, n_childs=1,dist_type="EUC_2D"):
    arr = np.array(G.nodes)
    childs = []

    for i in range(n_childs):
        np.random.shuffle(arr)
        child=[]
        cuts = np.random.randint(len(original), size=2)
        
        for i in range(np.min(cuts), np.max(cuts)):
            child.append(original[i])

        child = child + [i for i in arr if i not in child]
        child = individual(G,get_edge_list(child),dist_type)
        childs.append(child)
    return population(G,childs)

def create_next_gen(pop, target_size, mutation_rate=True,dist_type="EUC_2D"):
    #maybe add random individuals to crossbreed can improve performance
    if len(pop.individuals)==0: return generate_random_population(pop.G, dist_type, dist_type)
    gen = pop
    if mutation_rate==True: mutation_rate = 1/target_size

    n = math.floor((target_size - len(pop.individuals))/2)
    # n_inbred = math.floor(n/3)
    # n_random = n-n_inbred
    
    #inbreedings
    parents = gen.individuals
    for i in range(n):
        [p1, p2] =np.random.choice(parents, size=2)

        offspring1, offspring2 = crossover(p1.get_path(), p2.get_path(), mutation_rate)
        offspring1 = individual(pop.G, get_edge_list(offspring1), dist_type)
        offspring2 = individual(pop.G, get_edge_list(offspring2), dist_type)
        
        gen.append_individual(offspring1)
        gen.append_individual(offspring2)

    while len(gen.individuals)!=target_size: 
        gen.append_individual(generate_random_inividual(pop.G, dist_type))

    return gen

def node_positions(G):
    dictionary = nx.get_node_attributes(G,'coord')
    return dictionary

def get_graph_from_file(path):
    problem = tsplib95.load(path)
    return problem.get_graph()

def get_edge_list(array):
    if array == []: return array
    res=[]

    for i in range(len(array)-1):
        res.append((array[i],array[i+1]))

    res.append((array[-1],array[0]))

    return res

def euc_2d(node1, node2):
    return distance.euclidean(node1['coord'],node2['coord'])

def geo_dist(node1, node2):
    RRR = 6378.388
    [x1,y1] = node1['coord']
    [x2,y2] = node2['coord']
    
    deg = round(x1)
    min = x1 - deg
    latitude1 = np.pi * (deg + 5.0 * min / 3.0 ) / 180
    
    deg = round(y1)
    min = y1 - deg
    longitude1 = np.pi * (deg + 5.0 * min / 3.0 ) / 180
    
    deg = round(x2)
    min = x2 - deg
    latitude2 = np.pi * (deg + 5.0 * min / 3.0 ) / 180
    
    deg = round(y2)
    min = y2 - deg
    longitude2 = np.pi * (deg + 5.0 * min / 3.0 ) / 180
    
    q1 = np.cos(longitude1 - longitude2)
    q2 = np.cos(latitude1 - latitude2) 
    q3 = np.cos(latitude1 + latitude2)
    
    return int(RRR * np.arccos( 0.5*((1.0+q1)*q2 - (1.0-q1)*q3) ) + 1.0)

def get_fitness(G, edge_list, dist_type):
    fitness=0
    
    if dist_type=="EUC_2D":
        for pair in edge_list:
            fitness += euc_2d(G.nodes[pair[0]],G.nodes[pair[1]])
            
    elif dist_type=="GEO":
        for pair in edge_list:
            fitness += geo_dist(G.nodes[pair[0]],G.nodes[pair[1]])
    
    else:
        for pair in edge_list:
            fitness += G.edges[pair]['weight']
    return fitness

def get_path_from_edgelist(edge_list):
    return np.array([i[0] for i in edge_list])

def closest_neighbor_alg(G, dist_type):
    N = len(G.nodes)
    arr = np.arange(N) + 1
    np.random.shuffle(arr)
    visit = [np.random.choice(arr)]

    while True:
        next_city = 0
        min_fit = np.inf
        for i in arr:
            if i not in visit:
                edge = (i,visit[-1])
                fit = get_fitness(G,[edge],dist_type)
                if  fit < min_fit:  
                    min_fit = fit
                    next_city=i

        if next_city == 0: break
        visit.append(next_city)
    
    return np.array(visit)

def swap_positions_path(path, amount=1):
    for i in range(amount):
        pos = [0,0]
        while pos[0]==pos[1]:
            pos = np.random.randint(len(path), size=2)

        path[pos[0]], path[pos[1]] = path[pos[1]], path[pos[0]]
    return path

def tournament_selection(pop,n=5):
    parents = np.random.choice(pop.individuals, size=n)
    parents = sorted(parents, key=lambda agent: agent.fitness, reverse=True)
    return parents

def generate_random_path(G):
    arr=np.array(G.nodes)
    np.random.shuffle(arr)
    return arr

def plot_figure(G, edge_list, name="output.png", node_size=20, fig_size=10, title="fitness",dist_type="EUC_2D"):
    plt.clf()
    layout = node_positions(G)
    _, axs = plt.subplots(1, figsize=(fig_size,fig_size))
    # nx.draw_networkx(G, pos=layout, node_size=node_size, with_labels=False, edgelist=edge_list, ax=axs)
    nx.draw_networkx_edges(G, pos=layout, width=0.5,edgelist=edge_list, ax=axs)
    nx.draw_networkx_nodes(G, pos=layout, node_size=node_size,ax=axs)
    plt.axis("off")
    if title == "fitness":  plt.title(f"Fitness: {get_fitness(G,edge_list,dist_type):0.2f}, Distance type:{dist_type}")
    else:                   plt.title(title)
    plt.savefig(name, dpi=100)
    plt.close()
    
def load_gen(filename="last_gen"):
    with open(filename + ".pkl","rb") as inp:
        gen = pickle.load(inp)
    return gen

def load_individual(filename="individual"):
        with open(filename + ".pkl","rb") as inp:
            individual = pickle.load(inp)
        return individual


class individual:
    def __init__(self, g, edge_list, dist_type="EUC_2D"):
        self.G = g
        self.edge_list = edge_list
        self.fitness = get_fitness(g, edge_list, dist_type)
        self.dist_type = dist_type

    def set_edge_list(self, new_edge_list):
        self.edge_list = new_edge_list

    def get_path(self):
        return get_path_from_edgelist(self.edge_list)

    def set_path(self, path):
        self.edge_list = get_edge_list(path)

    def save_individual(self, name="best_individual"):
        with open(name+".pkl","wb") as outp:
            pickle.dump(self,outp,pickle.HIGHEST_PROTOCOL)
            
    def set_dist_type(self, dist_type):
        self.dist_type = dist_type

class population:
    def __init__(self, g, individuals):
        self.G = g
        self.individuals = individuals
    
    def get_mean_fitness(self):
        return np.mean([i.fitness for i in self.individuals])

    def get_fitnesses(self):
        return [i.fitness for i in self.individuals]

    def get_paths(self):
        return [i.get_path() for i in self.individuals]

    def get_fitness_data(self):
        fits = self.get_fitnesses()
        return np.argmin(fits), np.amin(fits),np.mean(fits)

    def append_individual(self, ind):
        self.individuals.append(ind)

    def save_gen(self, filename="last_gen"):
        with open(filename + ".pkl","wb") as outp:
            pickle.dump(self,outp,pickle.HIGHEST_PROTOCOL)
            
    def set_individuals_dists(self, dist_type):
        for i in self.individuals: i.set_dist_type(dist_type)

In [3]:
def write_gen_results(pop, filename, iter):
    if iter == 0:   file = open("/kaggle/working/results_" + filename[:-4] + ".txt","w")
    else:           file = open("/kaggle/working/results_" + filename[:-4] + ".txt","a")

    fitnessess = pop.get_fitnesses()
    best_in, _, _ = pop.get_fitness_data()
    file.write(f"\nIteration {iter} results: \nBest fitness = {fitnessess[best_in]}\n")

    for i in range(len(pop.individuals)):
        file.write(f"\nIndividual {i}, Fitness = {fitnessess[i]}")

def iteration(pop, n_next_gen):
    fits = pop.get_fitnesses()
    fits = np.argsort(fits)[:n_next_gen]

    res = population(pop.G,[pop.individuals[i] for i in fits])
    return res


def experiment(
filename = "bier127.tsp", 
dirname = "/kaggle/input/",
dirname_output="/kaggle/working/",
iterations=1000, 
interval = None, 
pop_size=20, 
n_next_gen = None, 
mutation_rate = None, 
outputs = True, 
initial_population=None,
check_progress=False,
increase_mutation=False,
print_mean=True,
keep_best=True,
dist_type="EUC2D"
):
    if n_next_gen==None: n_next_gen = math.floor(pop_size/4)
    if interval==None: interval = iterations/5
    if check_progress: check = interval/2

    print(f"TSP problem used:   {filename}")
    print(f"Num. iterations:    {iterations}")
    print(f"Interval:           {interval}")

    print(f"Population size:    {pop_size}")
    print(f"Mutation rate:      {mutation_rate}")

    if outputs:  print(f"Show outputs:       True")
    else:           print(f"Show outputs:       False")
    

    start = time.time()

    G = get_graph_from_file(dirname + filename)
    
    if mutation_rate==None: mutation_rate=1/len(G.nodes)
    if initial_population==None:
        #random generation
        pop = generate_random_population(G,pop_size,dist_type)

    else: pop = initial_population
    pop.set_individuals_dists(dist_type)
    if outputs:
        plot_figure(G, get_edge_list([]), title=filename, name=dirname_output + filename[:-4],dist_type=dist_type)

    bests=[]
    means=[]
    min_fit = np.inf

    for i in range(iterations):
        write_gen_results(pop,filename,i)
        pop.set_individuals_dists(dist_type)
        next_gen = iteration(pop, n_next_gen)
        
        i_best, best, mean = next_gen.get_fitness_data()
        bests.append(best)
        means.append(mean)

        if best < min_fit:
            min_fit = best
            best_individual = next_gen.individuals[i_best]
        
        pop = create_next_gen(next_gen, pop_size, mutation_rate=mutation_rate,dist_type=dist_type)
        
        if i==iterations-1: 
            print("Iteration",i, "; Best fitness:", best)
            break

        if check_progress and i%check==0: 

            if i!=0 and bests[i] >= bests[int(i-check)]: 
                if keep_best:
                    pop = split_path(G,next_gen.individuals[i_best].get_path(),pop_size-1, dist_type=dist_type)
                    pop.append_individual(next_gen.individuals[i_best])

                else: pop = split_path(G,next_gen.individuals[i_best].get_path(),pop_size,dist_type=dist_type)

        if i%interval==0:  
            if outputs:
                plot_figure(G, next_gen.individuals[i_best].edge_list, name=dirname_output+ filename[:-4] + "_"+ str(i))
            print("Iteration",i, "; Best generation fitness:", best)
        
    write_gen_results(pop,filename,i)
    pop.save_gen(f"{dirname_output}/best_{filename[:-4]}_gen_{i}")
    if outputs:
        plot_figure(G, pop.individuals[i_best].edge_list, name=dirname_output+ filename[:-4] + "_"+ str(i))

    took = time.time()-start
    print(f"Best overall fitness: {best_individual.fitness}")
    best_individual.save_individual(dirname_output + filename[:-4] + "_" +str(iterations)+ "_" +str(pop_size))
    # print(f"It took {took:.1f} seconds")

    plt.clf()
    plt.plot(bests, label="Best fitness")
    if print_mean: plt.plot(means, label="Mean")
    plt.legend()
    plt.xlabel("Generation")
    plt.ylabel("Fitness")
    plt.title(f"Population size: {pop_size}, {took:.1f} seconds")
    plt.savefig(dirname_output + filename[:-4] + "_progress_" + str(iterations) + "_popsize_" + str(pop_size) +".png")
    plt.close()

    return pop, best_individual


In [4]:
problem = tsplib95.load('/kaggle/input/tsp-problems/symmetric/eil51.tsp')
G  = problem.get_graph()              
dist_type = problem.as_dict()['edge_weight_type']
G.nodes[1]
closest_neighbor_alg(G, dist_type)

array([42, 19, 41, 13, 25, 14, 18,  4, 17, 37, 15, 44, 45, 33, 10, 49,  9,
       50, 16,  2, 29, 21, 34, 30, 39,  5, 38, 11, 32,  1, 22, 28, 31,  8,
       26,  7, 23, 48, 27, 51, 46, 12, 47,  6, 24, 43,  3, 20, 35, 36, 40])

# Symmetrical experiments

In [5]:
def experiment_symmetrical(l_iterations,l_pop_size):
    results = pd.DataFrame(columns=['filename', 'iterations', 'pop_size', 'global_best'])
    i=1
    for dirname, _, filenames in os.walk('/kaggle/input/tsp-problems/symmetric/'):
        for filename in filenames:
            print(f"{i}/{len(filenames)}")
            dir_out = '/kaggle/working/' + filename[:-4] + "/"
            os.mkdir(dir_out)
            problem = tsplib95.load(os.path.join(dirname, filename))
            G  = problem.get_graph()              
            dist_type = problem.as_dict()['edge_weight_type']
            
            init = closest_neighbor_alg(G, dist_type)
            for iterations in l_iterations:
                for pop_size in l_pop_size:
                    
                    pop = split_path(G,original=init,n_childs=pop_size-1,dist_type=dist_type)
                    pop.append_individual(individual(G,get_edge_list(init),dist_type))
                    
                    _,global_best = experiment(
                    filename=filename,
                    dirname=dirname,
                    dirname_output = dir_out,
                    iterations=iterations, 
                    interval=iterations/5,  
                    pop_size=pop_size,  
                    check_progress=True,
                    print_mean=False,
                    keep_best=True,
                    initial_population=pop,
                    dist_type=problem.as_dict()['edge_weight_type']
                    )
                    row = pd.DataFrame({'filename':filename,'iterations':iterations,'pop_size':pop_size,'global_best':global_best.fitness}, index=[0])
                    results = pd.concat([results,row], ignore_index=True)
                    results.to_csv("Symmetrical_results.csv")
                    print("\n")
            i+=1

# Asymmetrical experiments

In [6]:
 def experiment_asymmetrical(l_iterations,l_pop_size):
    results = pd.DataFrame(columns=['filename', 'iterations', 'pop_size', 'global_best'])
    i=1
    for dirname, _, filenames in os.walk('/kaggle/input/tsp-problems/asymmetric/'):
        for filename in filenames:
            print(f"{i}/{len(filenames)}")
            dir_out = '/kaggle/working/' + filename[:-5] + "/"
            os.mkdir(dir_out)
            problem = tsplib95.load(os.path.join(dirname, filename))
            G  = problem.get_graph()  
            dist_type = problem.as_dict()['edge_weight_type']
            
            init = closest_neighbor_alg(G, dist_type)
            for iterations in l_iterations:
                for pop_size in l_pop_size:
                    
                    pop = split_path(G,original=init,n_childs=pop_size-1,dist_type=dist_type)
                    pop.append_individual(individual(G,get_edge_list(init),dist_type))
                    
                    _,global_best = experiment(
                    filename=filename,
                    dirname=dirname,
                    dirname_output = dir_out,
                    iterations=iterations, 
                    interval=iterations/5, 
                    pop_size=pop_size,  
                    check_progress=True,
                    print_mean=False,
                    keep_best=True,
                    outputs=False, #always keep false for asymettric
                    initial_population=pop,
                    dist_type=problem.as_dict()['edge_weight_type']
                    )
                    row = pd.DataFrame({'filename':filename,'iterations':iterations,'pop_size':pop_size,'global_best':global_best.fitness}, index=[0])
                    results = pd.concat([results,row], ignore_index=True)
                    results.to_csv("Asymmetrical_results.csv")
                    print("\n")
            i+=1

In [ ]:
l_iterations=[100, 1000, 10000]
l_pop_size=[5, 10, 20, 40]

experiment_symmetrical(l_iterations,l_pop_size)
experiment_asymmetrical(l_iterations,l_pop_size)

# save everything in a ZIP

In [ ]:
!cd /kaggle/working/ & zip -jqr experiment_results.zip .